In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
inputs = ["test_image_data_0.parquet","test_image_data_1.parquet","test_image_data_2.parquet","test_image_data_3.parquet"]
#inputs = ["train_image_data_0.parquet","train_image_data_1.parquet","train_image_data_2.parquet","train_image_data_3.parquet"]

In [ ]:
DIR = "../input/bengaliai-cv19/"

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
import torch
import numpy as np
import cv2
from PIL import Image
class EvalDataset(Dataset):
    def __init__(self,df,transform=None,aug=None,norm=None):
        self.norm = norm
        self.df = df
        self.data = 255 - self.df.iloc[:, 1:].values.reshape(-1, 137, 236).astype(np.uint8)
        self.transform = transform
        self.aug = aug
        
    def __getitem__(self, idx):
        name = self.df.iloc[idx,0]
        image_arr = cv2.resize(self.data[idx],(128,128))
        image_arr = image_arr/255.0
        if self.norm != None:
            mean = self.norm['mean']
            std = self.norm['std']
            image_arr = (image_arr -  mean)/std
        image_arr = torch.from_numpy(image_arr.astype(np.float32))
        return image_arr.unsqueeze(0).repeat(3,1,1),name


    def __len__(self):
        return len(self.data)

In [ ]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
from torchvision import models
import torch.nn as nn
# Easier to split stuff up and backpropagate
class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    model = models.resnet34()
    self.model = nn.Sequential(*list(model.children())[:-1])#chop off last layer
    self.fc_g = nn.Sequential(nn.Linear(512,200), nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(200,168))
    self.fc_v = nn.Sequential(nn.Linear(512,200), nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(200,11))
    self.fc_c = nn.Sequential(nn.Linear(512,200), nn.ReLU(), nn.Dropout(p=0.2), nn.Linear(200,7))
  def forward(self,x):
    x = self.model(x)
    # print(x.shape)
    x = torch.flatten(x,1)
    g = self.fc_g(x)
    v = self.fc_v(x)
    c = self.fc_c(x)
    return g,v,c

In [ ]:
def kaggle_out(model1,model2,model3,dataloader,device):
    model1.eval()
    model2.eval()
    model3.eval()
    grapheme_output = []
    vowel_output = []
    consonant_output = []
    img_id_ls = []
    for data in dataloader:
        inputs = data[0]
        image_id = data[1]
        inputs = inputs.to(device)
        with torch.no_grad():
            g1,v1,c1 = model1(inputs)
            g2,v2,c2 = model2(inputs)
            g3,v3,c3 = model3(inputs)
            g = g1+g2+g3
            v = v1+v2+v3
            c = c1+c2+c3
            grapheme_preds = g.argmax(dim=1)
            vowel_preds = v.argmax(dim=1)
            consonant_preds = c.argmax(dim=1)
        for i in range(len(image_id)):
            img_id_ls.append(image_id[i])
            grapheme_output.append(grapheme_preds[i].cpu().detach().item())
            vowel_output.append(vowel_preds[i].cpu().detach().item())
            consonant_output.append(consonant_preds[i].cpu().detach().item())
    return img_id_ls,grapheme_output,vowel_output,consonant_output

In [ ]:
results = []
mean = 13.4/255
std = 40.8/255
model1 = MyModel()
model1.load_state_dict(torch.load('../input/cv-resnets/model_epoch34.pt'))
model1.to(device)

model2 = MyModel()
model2.load_state_dict(torch.load('../input/cv-resnets/model_epoch69.pt'))
model2.to(device)

model3 = MyModel()
model3.load_state_dict(torch.load('../input/cv-resnets/model_epoch104.pt'))
model3.to(device)
for inp in inputs:
    df = pd.read_parquet(DIR+inp)
    kaggle_dataset = EvalDataset(df,norm={'mean':mean,'std':std})
    dl = DataLoader(kaggle_dataset,batch_size=128,num_workers=4)
    img,gra,vow,con = kaggle_out(model1,model2,model3,dl,device)
    for i in range(len(img)):
        results.append((f"{img[i]}_grapheme_root",gra[i]))
        results.append((f"{img[i]}_vowel_diacritic",vow[i]))
        results.append((f"{img[i]}_consonant_diacritic",con[i]))
df2 = pd.DataFrame(results, columns=["row_id", "target"])

In [ ]:
sub = df2

In [ ]:
sub.to_csv("submission.csv", index=False)

In [ ]:
sub